In [1]:
import json
import os

In [2]:
def read_file_and_convert_keys(file_path: str) -> list:
    # Open the file and read the content
    with open(file_path, 'r') as file:
        file_content = file.read().strip()

    # Split the file content by new line
    lines = file_content.split('\n')

    # Map each line to a tuple of (E, N)
    result = []
    for line in lines:
        E, N = line.split(',')
        result.append((int(E.strip()), int(N.strip())))

    return result

In [3]:
def bigint_to_array(n: int, k: int, x: int) -> list:
    mod = 1
    for _ in range(n):
        mod *= 2

    ret = []
    x_temp = x
    for _ in range(k):
        ret.append(x_temp % mod)
        x_temp //= mod  # Use integer division to ensure x_temp remains an integer

    return ret

In [4]:
def generate_json_efficient(m, exp, publicKeyExp, publicKeyM, sign, hashed, nkeys):
    # Prepare the publicKey_array, convert numbers to strings
    publicKey_array = [[str(num) for num in bigint_to_array(64, 32, publicKeyExp)],
                       [str(num) for num in bigint_to_array(64, 32, publicKeyM)]]
    
    # Prepare the publicKeys_array, convert numbers to strings
    publicKeys_array = []
    publicKeys_array.append(publicKey_array)
    for i in range(nkeys-1):
        publicKeys_array.append([
            [str(num) for num in bigint_to_array(64, 32, exp[i])],  # Exponent as string
            [str(num) for num in bigint_to_array(64, 32, m[i])]     # Modulus as string
        ])

    # Convert sign and hashed using bigint_to_array, convert numbers to strings
    sign_array = [str(num) for num in bigint_to_array(64, 32, sign)]
    hashed_array = [str(num) for num in bigint_to_array(64, 4, hashed)]

    # Structure the result as a dictionary matching the JSON format
    result = {
        "publicKeys": publicKeys_array,
        "sign": sign_array,
        "hashed": hashed_array,
        "publicKey": publicKey_array
    }

    # Convert to JSON string
    return json.dumps(result, indent=4)

In [5]:
def generate_json_strong(m, exp, publicKeyExp, publicKeyM, sign, hashed, nkeys):

    publicKey_array = [[str(num) for num in bigint_to_array(64, 32, publicKeyExp)],
                       [str(num) for num in bigint_to_array(64, 32, publicKeyM)]]
    
    # Prepare the publicKeys_array, convert numbers to strings
    publicKeys_array = []
    publicKeys_array.append(publicKey_array)
    for i in range(nkeys-1):
        publicKeys_array.append([
            [str(num) for num in bigint_to_array(64, 32, exp[i])],  # Exponent as string
            [str(num) for num in bigint_to_array(64, 32, m[i])]     # Modulus as string
        ])

    # Convert sign and hashed using bigint_to_array, convert numbers to strings
    sign_array = [str(num) for num in bigint_to_array(64, 32, sign)]
    hashed_array = [str(num) for num in bigint_to_array(64, 4, hashed)]

    # Structure the result as a dictionary matching the JSON format
    result = {
        "publicKeys": publicKeys_array,
        "sign": sign_array,
        "hashed": hashed_array
    }

    # Convert to JSON string
    return json.dumps(result, indent=4)

In [6]:
def write_json_input_files(m, exp, publicKeyExp, publicKeyM, sign, hashed, nkeys):
    # Specify the folder where you want to store the file
    strong_folder_path = "strong_inputs_folder"
    efficient_folder_path = "efficient_inputs_folder"
    
    # Create the folders if they don't not exist
    os.makedirs(strong_folder_path, exist_ok=True)
    os.makedirs(efficient_folder_path, exist_ok=True)
    
    # Dynamically create the file name based on the size of the m array
    strong_input_file_name = f'output_strong_{nkeys}.json'
    efficient_input_file_name = f'output_efficient_{nkeys}.json'
    
    # Construct the full file path with folder
    strong_output_file_path = os.path.join(strong_folder_path, strong_input_file_name)
    efficient_output_file_path = os.path.join(efficient_folder_path, efficient_input_file_name)
    
    # Generate the JSON
    efficient_json_data = generate_json_efficient(m, exp, publicKeyExp, publicKeyM, sign, hashed, nkeys)
    strong_json_data = generate_json_strong(m, exp, publicKeyExp, publicKeyM, sign, hashed, nkeys)
    
    # Write the JSON to a file directly
    with open(efficient_output_file_path, "w") as efficient_json_file:
        efficient_json_file.write(efficient_json_data)
    print(f"Efficient Group Signature JSON data with {nkeys} keys has been written to {efficient_output_file_path}")
    
    with open(strong_output_file_path, "w") as strong_json_file:
        strong_json_file.write(strong_json_data)
    print(f"Strong Group Signature JSON data with {nkeys} keys has been written to {strong_output_file_path}")

# What we do is read all the public keys, signature and hash. Then we transfrom them into readable inputs and generate an input file to be provided to the proof generation protocol.
# We will do inputs for 50, 500, 5000 and 50000 public keys.

In [7]:
file_path = "inputKeys_fake.txt"
public_keys = read_file_and_convert_keys(file_path)

# Separate exponents and modulus into two arrays
exp = [key[0] for key in public_keys]  # Extract the first element (E)
m = [key[1] for key in public_keys]  # Extract the second element (N)

In [8]:
# Function to read the JSON signature file and assign the values to variables
def read_json_file(file_path):
    # Open and load the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)
    data = json.loads(data)
    # Extract the values and assign to variables
    N = data["N"]
    E = data["E"]
    sign = data["sign"]
    hashed = data["hashed"]
    return E, N, sign, hashed

In [9]:
signature_path = "signature.json"  # The path to your JSON file
publicKeyExp, publicKeyM, sign, hashed = read_json_file(signature_path)


#m = [26877283975388232002065546470999785257065668437121277626422898829468295994425002534123931915945559617344634299463672464881786335501998933229817362492919530719356197980244178171682568314630456704843964235701169272840827403226569155673809960150900311743961237789646475480998560495177043349952420160197708953986181525637253074713500260384690148853022580356324720561179472725807442763843755515338818629917380201208369993837646911190634717770914200111131983791094934970474752204714209363619762523501868989404033152536987791077018610508510651331541408664242381532359216354015898457615549908800129418374861154322861098417303,
    #27333278531038650284292446400685983964543820405055158402397263907659995327446166369388984969315774410223081038389734916442552953312548988147687296936649645550823280957757266695625382122565413076484125874545818286099364801140117875853249691189224238587206753225612046406534868213180954324992542640955526040556053150097561640564120642863954208763490114707326811013163227280580130702236406906684353048490731840275232065153721031968704703853746667518350717957685569289022049487955447803273805415754478723962939325870164033644600353029240991739641247820015852898600430315191986948597672794286676575642204004244219381500407]
#exp = [65537, 65537]
#publicKeyExp = 65537
#publicKeyM = 26877283975388232002065546470999785257065668437121277626422898829468295994425002534123931915945559617344634299463672464881786335501998933229817362492919530719356197980244178171682568314630456704843964235701169272840827403226569155673809960150900311743961237789646475480998560495177043349952420160197708953986181525637253074713500260384690148853022580356324720561179472725807442763843755515338818629917380201208369993837646911190634717770914200111131983791094934970474752204714209363619762523501868989404033152536987791077018610508510651331541408664242381532359216354015898457615549908800129418374861154322861098417303
#sign = 19970770434011356420695616570275995588733851736092916486111712132583641231522399849078260379017479208895803667652431366593883985802635999227638120171560060271785507252576191070096993051229216860066470378690422171938259912306738962186526007146103769150108002059810205462974903424990722607936274237920270872702180029184401091695395600924057276873789754646380608203609851792334985336197032772780502144607311769527475415342699750160560754943907331155361187938339206488970924682929057116994252391614878690800996605435792148903886765163146205141249146933886278846746712104071544235211937440019023711390224571884634365863432
#hashed = 72155939486846849509759369733266486982821795810448245423168957390607644363272

In [10]:
#Write 5 keys
write_json_input_files(m[:5], exp[:5], publicKeyExp, publicKeyM, sign, hashed, 5)

#Write 10 keys
write_json_input_files(m[:10], exp[:10], publicKeyExp, publicKeyM, sign, hashed, 10)

#Write 50 keys
write_json_input_files(m[:20], exp[:20], publicKeyExp, publicKeyM, sign, hashed, 20)

#Write 50 keys
write_json_input_files(m[:50], exp[:50], publicKeyExp, publicKeyM, sign, hashed, 50)

#Write 500 keys
write_json_input_files(m[:500], exp[:500], publicKeyExp, publicKeyM, sign, hashed, 500)

#Write 5000 keys
write_json_input_files(m[:5000], exp[:5000], publicKeyExp, publicKeyM, sign, hashed, 5000)

#Write 50000 keys
write_json_input_files(m[:50000], exp[:50000], publicKeyExp, publicKeyM, sign, hashed, 50000)

Efficient Group Signature JSON data with 5 keys has been written to efficient_inputs_folder\output_efficient_5.json
Strong Group Signature JSON data with 5 keys has been written to strong_inputs_folder\output_strong_5.json
Efficient Group Signature JSON data with 10 keys has been written to efficient_inputs_folder\output_efficient_10.json
Strong Group Signature JSON data with 10 keys has been written to strong_inputs_folder\output_strong_10.json
Efficient Group Signature JSON data with 20 keys has been written to efficient_inputs_folder\output_efficient_20.json
Strong Group Signature JSON data with 20 keys has been written to strong_inputs_folder\output_strong_20.json
Efficient Group Signature JSON data with 50 keys has been written to efficient_inputs_folder\output_efficient_50.json
Strong Group Signature JSON data with 50 keys has been written to strong_inputs_folder\output_strong_50.json
Efficient Group Signature JSON data with 500 keys has been written to efficient_inputs_folder\ou